# LRT Request Analysis

## Functionalities
- Analysis of RPCs and queries of LRT requests.

## Input
Log files are read from a directory in `../data`. This directory is assumed to have the following structure:
```
logs/
  [node-1]/
    *_service*.tar.gz
    ...
    apigateway*.tar.gz
    ...
    loadgen.tar.gz
  ...
  [node-n]/
    *_service*.tar.gz
    ...
    apigateway*.tar.gz
    ...
    loadgen.tar.gz
```

## Notebook Configuration

In [ ]:
########## GENERAL
# Name of the directory in `../data`
EXPERIMENT_DIRNAME = "BuzzBlogBenchmark_[TIMESTAMP]"

########## LATENCY
# Latency threshold (in sec)
LRT_REQUEST_LATENCY_THRESHOLD = 1

## Notebook Setup

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import os
import pandas as pd
import sys
import warnings
warnings.filterwarnings("ignore")

sys.path.append(os.path.abspath(os.path.join("..")))
from utils.utils import *

experiment_dirpath = os.path.join(os.path.abspath(""), "..", "data", EXPERIMENT_DIRNAME)

## Log Parsing & Processing

In [ ]:
# Build data frames
requests = pd.concat([df[2] for df in get_loadgen_df(experiment_dirpath)])
rpc = pd.concat([df[2] for df in get_rpc_df(experiment_dirpath)])
query = pd.concat([df[2] for df in get_query_df(experiment_dirpath)])

## Analysis of LRT Requests

In [ ]:
lrt_requests = requests[(requests["latency"] > LRT_REQUEST_LATENCY_THRESHOLD)]
for lrt_request in lrt_requests.to_dict("records"):
    print(lrt_request)
    print("Request ID: %s" % lrt_request["request_id"])
    print("  Type: %s" % lrt_request["type"])
    print("  RPCs:")
    for lrt_request_rpc in rpc[(rpc["request_id"] == lrt_request["request_id"])].to_dict("records"):
        print("    %s - %s" % (lrt_request_rpc["function"], lrt_request_rpc["latency"]))
    print("  Queries:")
    for lrt_request_query in query[(query["request_id"] == lrt_request["request_id"])].to_dict("records"):
        print("    %s - %s" % (lrt_request_query["dbname"] + ":" + lrt_request_query["type"], lrt_request_query["latency"]))